# Import các thư viện cần thiết


In [1]:
import warnings
import os
from tqdm import tqdm
from typing import List
import pandas as pd
from unidecode import unidecode
import plotly.express as px
import unicodedata
import math
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")


# from langdetect import detect
# from langdetect import DetectorFactory

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

# 2. Tiền xử lý dữ liệu thống kê từ video


Đọc dữ liệu thống kê video thành DataFrame.


In [2]:
# raw_df1 = pd.read_csv("data/old_users_amthuc.csv", low_memory=False)
# raw_df1.info()

In [3]:
# raw_df2 = pd.read_csv("data/new_users_amthuc_1_handmade.csv", low_memory=False)
# raw_df2.info()

In [4]:
# raw_df3 = pd.read_csv("data/new_usrs_amthuc_250319_0033.csv", low_memory=False)
# raw_df3.info()

In [5]:
video_info_df = pd.read_csv("data/final_raw_videos.csv", low_memory=False)
video_info_df.info()
# video_info_df = pd.concat([raw_df1, raw_df2, raw_df3], ignore_index=True)
# video_info_df.info()
# video_info_df.to_csv('data/final_raw_videos.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254197 entries, 0 to 254196
Columns: 185 entries, Unnamed: 0 to video.zoomCover.960
dtypes: float64(97), int64(2), object(86)
memory usage: 358.8+ MB


In [6]:
# save_df = video_info_df[~video_info_df['author.uniqueId'].isna()]
# save_list = list(save_df['author.uniqueId'].unique())
# len(save_list)


# def write_list_to_file(file_path, data_list):
#     try:
#         with open(file_path, "w", encoding="utf-8") as file:
#             for item in data_list:
#                 file.write(str(item) + "\n")
#         print(f"Successfully wrote {len(data_list)} items to {file_path}")
#     except Exception as e:
#         print(f"Error writing to file: {e}")


# write_list_to_file("data/lists/users_from_new_old_csv.txt", save_list)

Loại bỏ các cột có tỷ lệ thiếu dữ liệu lớn hơn 50%


In [7]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display first 5 rows
missing_ratio.head()

video.zoomCover.960                       1.0
user.commerceUserInfo.downLoadLink.ios    1.0
video.encodeUserTag                       1.0
poi.country                               1.0
poi.fatherPoiId                           1.0
dtype: float64

In [8]:
# Remove columns with missing ratios greater than 0.50
video_info_df = video_info_df.dropna(
    axis='columns', thresh=0.50 * len(video_info_df)
)

In [9]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display the missing ratio
for column, ratio in missing_ratio[:5].items():
    print(f"{column:50}:{ratio:8.2%}")

video.claInfo.originalLanguageInfo.languageID     :  44.36%
video.claInfo.originalLanguageInfo.languageCode   :  44.36%
video.claInfo.originalLanguageInfo.language       :  44.36%
video.claInfo.originalLanguageInfo.canTranslateRealTimeNoCheck:  44.36%
video.claInfo.captionsType                        :  44.36%


In [10]:
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254197 entries, 0 to 254196
Data columns (total 88 columns):
 #   Column                                                          Non-Null Count   Dtype  
---  ------                                                          --------------   -----  
 0   Unnamed: 0                                                      254197 non-null  int64  
 1   stats.diggCount                                                 254197 non-null  int64  
 2   CategoryType                                                    253333 non-null  float64
 3   author.commentSetting                                           253333 non-null  float64
 4   author.downloadSetting                                          253333 non-null  float64
 5   author.duetSetting                                              253333 non-null  float64
 6   author.ftc                                                      253333 non-null  object 
 7   author.id                             

Vì các cột có tên bắt đầu với `stats.*` chứa cùng thông tin với các cột có tên bắt đầu với `statsV2.*`, nhưng không có thông tin về `repostCount` như `statsV2.*`. Nên ta sẽ loại bỏ các cột có tên bắt đầu với `stats.*`.


In [11]:
# Remove columns starting with "stats."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("stats.")]
]

# video_info_df.info()

Xóa các cột bắt đầu với `video.claInfo.originalLanguageInfo.*` vì chúng chứa thông tin không cần thiết.


In [12]:
# Remove columns starting with "video.claInfo.originalLanguageInfo."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("video.claInfo.originalLanguageInfo.")]
]

# video_info_df.info()

Chuyển timestamp sang datetime và chuyển sang giờ VN.


In [13]:
# Chuyển timestamp sang dạng datetime UTC
video_info_df["createTime"] = pd.to_datetime(
    video_info_df["createTime"], unit="s", utc=True)

# Chuyển sang múi giờ Việt Nam (UTC+7)
video_info_df["createTime"] = video_info_df["createTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")

Tạo một cột chứa danh sách các `hashtag` được trích xuất từ mô tả video. Và tính số lượng hashtag trong mỗi video.


In [14]:
# Replace missing values in "desc" column with an empty string
video_info_df["desc"] = video_info_df["desc"].fillna("")
video_info_df["desc"] = video_info_df["desc"].astype(str)
video_info_df["desc"] = video_info_df["desc"].str.strip()

# Create a new column for the hashtags
# and the number of hashtags in each video
video_info_df["hashtags"] = [""] * len(video_info_df)
video_info_df["num_hashtags"] = [0] * len(video_info_df)

# Extract hashtags from the "desc" column
# and Get the number of hashtags in each video
for index in tqdm(range(len(video_info_df))):
    # Get the description of the video
    description = video_info_df.loc[index, "desc"].strip().lower()

    if description:
        # # Remove emojis
        # description = description.encode('ascii', 'ignore').decode('ascii')
        # Chuyển tiếng Việt có dấu thành không dấu
        description = unidecode(description)

        # Add a space before all "#" characters
        description = description.replace("#", " #")

        # Find all strings starting with "#" and followed by a word
        hashtags = [word[1:] for word in description.split()
                    if word.startswith("#")]

        # Extract hashtags from the description
        video_info_df.loc[index, "hashtags"] = ",".join(hashtags).strip()

        # Get the number of hashtags
        video_info_df.loc[index, "num_hashtags"] = len(hashtags)
    else:
        video_info_df.loc[index, "hashtags"] = ""
        video_info_df.loc[index, "num_hashtags"] = 0

100%|██████████| 254197/254197 [00:36<00:00, 6903.65it/s]


In [15]:
# Bước 1: Chuyển hashtags thành danh sách
video_info_df["hashtags"] = video_info_df["hashtags"].apply(
    lambda x: x.split(",") if x else [])

### Chọn ra một số cột để kiểm tra thông số.


In [16]:
selected_cols = {
    'id': 'id',
    'author.uniqueId': 'username',
    "statsV2.playCount": "views",  # Lượt xem video
    "statsV2.diggCount": "likes",  # Lượt thích video
    "statsV2.commentCount": "comments",  # Lượt bình luận video
    "statsV2.shareCount": "shares",  # Lượt chia sẻ video
    'statsV2.collectCount': "collects",
    'authorStats.diggCount': "user.likes",
    'authorStats.followerCount': "user.followers",
    'authorStats.followingCount': "user.followings",
    # 'authorStats.friendCount': "user.friends",
    'authorStats.heartCount': "user.hearts",
    'authorStats.videoCount': "user.videos",
    'hashtags': 'hashtags',
    'createTime': 'createTime',
    'desc': 'desc',
    'textLanguage': 'textLang'
}
stats_df = video_info_df[list(selected_cols.keys())].rename(
    columns=selected_cols)
stats_df.sample(5)

,id,username,views,likes,comments,shares,collects,user.likes,user.followers,user.followings,user.hearts,user.videos,hashtags,createTime,desc,textLang
225739,7.351051e+18,kem19995,2100000.0,171700.0,249.0,406.0,2218.0,1156.0,4100000.0,5.0,200400000.0,998.0,[kemtiktok],2024-03-27 21:30:48+07:00,Anh chị chơi kỳ 😆#kemtiktok,vi
135931,7.404950e+18,alexiaeatsboba,344300.0,24700.0,56.0,679.0,1358.0,7202.0,973700.0,280.0,80400000.0,333.0,"[eatwithme, mukbang, mukbangeatingshow, eating...",2024-08-20 03:29:56+07:00,I couldn’t say no to this weeks lineup 😍😋 love...,en
147357,7.445635e+18,hai88tk,118100.0,2527.0,74.0,65.0,71.0,5.0,55500.0,11.0,2600000.0,403.0,"[giadinh, anuong, danong, vochong, vochongson,...",2024-12-07 18:45:47+07:00,Quá sai lầm rồi #giadinh #anuong #danong #voch...,vi
164364,7.474232e+18,dullmis,838400.0,47300.0,223.0,1585.0,1399.0,3353.0,607900.0,274.0,31500000.0,865.0,"[xuhuong, mukbang, dullmis, ancungtiktok]",2025-02-22 20:17:26+07:00,Mọi người còn nhớ xôi chiên hong #xuhuong #muk...,vi
53799,7.307859e+18,longquandui,226800.0,3983.0,179.0,955.0,1077.0,43.0,135500.0,84.0,4200000.0,827.0,"[longquandui, ancungtiktok, learnontiktok, vir...",2023-12-02 12:05:08+07:00,Trà sữa đất nung Vân Nam đã có ở Tân Phú rồi n...,vi


Drop các dòng có thiếu hơn 50% số cột.


In [17]:
stats_df["missing_count"] = stats_df.isnull().sum(axis=1)
# Xóa các dòng có hơn 50% giá trị bị thiếu
threshold = stats_df.shape[1] * 0.5

# Lọc bỏ các dòng có số lượng giá trị thiếu vượt quá ngưỡng
stats_df = stats_df[stats_df["missing_count"] <= threshold].drop(
    columns=["missing_count"])
stats_df.shape

(253333, 16)

# 3. Lọc lại danh sách user


In [18]:
print("Number of users: ", stats_df['username'].nunique())

Number of users:  841


Lọc các video đăng trong năm 2025 (loại bỏ người dùng không hoạt động).


In [93]:
# time_thres = pd.to_datetime("2025-01-01").tz_localize("Asia/Ho_Chi_Minh")
# users_in_2025 = stats_df[stats_df['createTime']
#                          >= time_thres]['username'].unique()
# filt_stats_df = stats_df[stats_df['username'].isin(users_in_2025)]

time_thres = pd.to_datetime("2025-01-01").tz_localize("Asia/Ho_Chi_Minh")
filt_stats_df = stats_df[stats_df['createTime'] >= time_thres]
# filt_stats_df = stats_df

print("Number of users: ", filt_stats_df['username'].nunique())

Number of users:  771


### Các thông số thống kê theo từng user về lượt xem và độ tương tác.


In [94]:
# Lọc lại các cột quan trọng liên quan đến user và các chỉ số tương tác của video
df_selected = filt_stats_df[[
    # "author.id",  # ID của user
    "username",  # Tên hiển thị của user
    "views",  # Lượt xem video
    "likes",  # Lượt thích video
    "comments",  # Lượt bình luận video
    "shares",  # Lượt chia sẻ video
    'collects',
    'user.followers',
    'user.likes'
]]

# Kiểm tra dữ liệu bị thiếu
df_selected = df_selected.dropna()

# Tổng hợp dữ liệu theo user
user_stats_df_0 = df_selected.groupby(["username"]).agg(
    total_views=("views", "sum"),
    avg_views=("views", "mean"),
    total_likes=("likes", "sum"),
    avg_likes=("likes", "mean"),
    total_comments=("comments", "sum"),
    total_shares=("shares", "sum"),
    total_collects=("collects", "sum"),
    total_videos=("views", "count"),  # Số lượng video của user
    followers=('user.followers', 'max'),
    likes=('user.likes', 'max'),
).reset_index()

user_stats_df_0 = user_stats_df_0.rename(columns={'username': 'username'})
# Tính Engagement Rate (ER) trung bình
user_stats_df_0["engagement_rate"] = (user_stats_df_0["total_likes"] + user_stats_df_0["total_comments"] +
                                      user_stats_df_0["total_shares"]+user_stats_df_0["total_collects"]) / user_stats_df_0["total_views"]
user_stats_df_0["total_engagement"] = (user_stats_df_0["total_likes"] + user_stats_df_0["total_comments"] +
                                       user_stats_df_0["total_shares"]+user_stats_df_0["total_collects"])
# user_stats_df_0["avg_engagement"] = user_stats_df_0["total_engagement"] / \
#     user_stats_df_0["total_videos"]
# Loại bỏ những user có tổng lượt xem bằng 0 (tránh lỗi chia cho 0)
user_stats_df_0 = user_stats_df_0[user_stats_df_0["total_views"] > 0]
display(user_stats_df_0.sample(5))
# display(user_stats_df_0.describe())

,username,total_views,avg_views,total_likes,avg_likes,total_comments,total_shares,total_collects,total_videos,followers,likes,engagement_rate,total_engagement
471,narrmulayofficial,14166400.0,7.083200e+05,907662.0,45383.100000,6972.0,7640.0,26026.0,20,300300.0,19.0,0.066940,948300.0
260,giadinhzozo,52805900.0,8.517081e+05,2494442.0,40232.935484,10892.0,22421.0,64717.0,62,661800.0,5567.0,0.049094,2592472.0
374,kun_thich_nauan,27628043.0,4.763456e+05,392189.0,6761.879310,4283.0,37496.0,65481.0,58,21800.0,3627.0,0.018078,499449.0
556,phuongoanh.daily,59900157.0,5.300899e+05,1372425.0,12145.353982,29800.0,38824.0,47456.0,113,939400.0,5118.0,0.024850,1488505.0
94,bayashi.tiktok,166989800.0,6.679592e+06,10321700.0,412868.000000,76420.0,241226.0,513957.0,25,55400000.0,1192.0,0.066790,11153303.0


### Phần trăm video có caption bằng tiếng Việt


In [154]:
# Tổng số video theo từng user
total_videos = stats_df.groupby('username')['textLang'].count()

# Số video có textLang là 'vi'
vi_videos = stats_df[stats_df['textLang'] == 'vi'].groupby('username')[
    'textLang'].count()

# Ghép hai bảng, thay NaN bằng 0 (nếu user không có video tiếng Việt)
percent_vi = (vi_videos / total_videos).fillna(0).reset_index()

# Đổi tên cột
percent_vi.columns = ['username', 'vi_video_percentage']
percent_vi.sort_values('vi_video_percentage', ascending=False)

,username,vi_video_percentage
0,.banhsukem,1.0
463,lyluclac28,1.0
192,chuyennhauttinh,1.0
191,chuvuongdimorua,1.0
190,chutunghamvui66988,1.0
...,...,...
35,amthucth841,0.0
206,crustys_mtl,0.0
445,lifeofcian,0.0
446,lilah.eats,0.0


### Khoảng thời gian giữa 2 bài đăng của user mà 75% bài đăng thoả ngưỡng đó.

Tức là 75% các video của có khoảng cách giữa các lần đăng dưới ngưỡng này.


In [155]:
select_df = filt_stats_df
select_df = select_df.sort_values(by=["username", "createTime"])

# Tính khoảng cách thời gian giữa các video liên tiếp của mỗi user
select_df["time_diff"] = select_df.groupby("username")["createTime"].diff()

# Xử lý trường hợp user chỉ có 1 video, không tính được diff
select_df["time_diff"] = select_df["time_diff"].fillna(pd.Timedelta(seconds=0))

# Tính trung bình khoảng cách giữa các video cho mỗi user
frequency_df = select_df.groupby("username")["time_diff"].quantile(
    0.75).dt.total_seconds() / 3600 / 24  # Đổi ra ngày

# Nếu chỉ có 1 video, gán giá trị 'inf' cho 'days_between_videos'
frequency_df = frequency_df.reset_index(name="days_between_videos")

# Xử lý trường hợp user có 1 video
single_video_users = frequency_df[frequency_df["days_between_videos"] == 0]
frequency_df.loc[single_video_users.index, "days_between_videos"] = float(
    'inf')  # Gán giá trị 'inf' cho những người chỉ có 1 video
frequency_df['frequency'] = 1 / frequency_df['days_between_videos']

# Kiểm tra kết quả
frequency_df.head()

,username,days_between_videos,frequency
0,.banhsukem,2.066962,0.483802
1,05061993chomthao,8.379497,0.119339
2,1108ng,1.017078,0.983209
3,1phutsaigon,1.057795,0.945363
4,30days__giamcanlanhmanh,0.813108,1.229849


### Tính phần trăm cái bài đăng có liên quan đến ẩm thực (dựa trên hashtag)


Danh sách các hashtag, số video tương ứng và số user sử dụng hashtag đó.


In [156]:
# Tách hashtag thành từng dòng riêng biệt
exploded_df = stats_df.explode("hashtags")

# Đếm số lượng video chứa mỗi hashtag
hashtag_video_counts = exploded_df["hashtags"].value_counts().reset_index()
hashtag_video_counts.columns = ["hashtags", "videos_count"]

# Đếm số tài khoản sử dụng mỗi hashtag (số user duy nhất)
hashtag_user_counts = exploded_df.groupby(
    "hashtags")["username"].nunique().reset_index()
hashtag_user_counts.columns = ["hashtags", "users_count"]

# Gộp cả 2 thông tin vào cùng 1 DataFrame
hashtags_df = hashtag_video_counts.merge(hashtag_user_counts, on="hashtags")
hashtags_df["avg_videos_per_user"] = hashtags_df["videos_count"] / \
    hashtags_df["users_count"]
hashtags_df = hashtags_df.sort_values('users_count', ascending=False)
display(hashtags_df)
# px.bar(hashtags_df.head(50), x='users_count', y='hashtags')

,hashtags,videos_count,users_count,avg_videos_per_user
0,ancungtiktok,72397,506,143.077075
14,,9293,468,19.856838
1,xuhuong,55936,466,120.034335
2,learnontiktok,51133,415,123.212048
34,thanhthoiluottet,5918,384,15.411458
...,...,...,...,...
44474,xuhaoxaocuonghong,1,1,1.000000
44475,ensaio.,1,1,1.000000
44476,buncavietnam,1,1,1.000000
44477,rumble,1,1,1.000000


Lọc ra các hashtag có liên quan đến ẩm thực.


In [157]:
food_keywords = [
    "food", "eat", "drink", "chef", "cook", "cooking", "recipe", "baking", "buffet", "steak", "grill", "vegan",
    "anvat",  "anuong",  "angi",  "anngon", "doan", "ancung", 'nauan', 'anchay',
    "douong", "nuocuong",
    "monngon", 'monchay',
    "ancom", "anvat",  "moncom", "buacom", 'comnha', 'buacom', 'comtam', 'comchay',
    "hutieu", "banhmi", "haisan", 'micay', 'mituongden', 'mycay', 'mytuongden',
    "trasua", "nuoc", "caphe", "sinhto", "ruou", 'latte', 'cacao', 'caffe', 'bun', 'banh',
    "amthuc",  "nhahang", "thucdon", "dacsan", "mukbang",
]

# food_keywords = [
#     "food", "eat", "drink", "chef", "cook", "cooking", "recipe", "baking", "buffet", "steak", "grill", "vegan",
#     "amthuc", "an",  "nhahang", "mon", "ngon", "uong", "thucdon", "dacsan", "tiem",
#     # "ancom", "anvat",  "moncom", "buacom", 'comnha', 'buacom',
#     "hutieu", "banhmi", "haisan", "com", 'bun', 'canh',
#     "tra", "sua", "nuoc", "caphe", "sinhto", "ruou", 'latte', 'cacao', 'caffe',
#     'banh',
# ]

food_related_hashtags_df = hashtags_df[hashtags_df["hashtags"].apply(
    lambda x: any(kw in x for kw in food_keywords))]
food_related_hashtags_df = food_related_hashtags_df.sort_values(
    'users_count', ascending=False, ignore_index=True)

# with pd.option_context("display.max_rows", None):
display(food_related_hashtags_df)

,hashtags,videos_count,users_count,avg_videos_per_user
0,ancungtiktok,72397,506,143.077075
1,mukbang,22170,248,89.395161
2,reviewanngon,29737,239,124.422594
3,food,15632,225,69.475556
4,anvat,5306,206,25.757282
...,...,...,...,...
11253,tulambanhmi,1,1,1.000000
11254,banheiradegelo,1,1,1.000000
11255,steaktartare,1,1,1.000000
11256,takafoods,1,1,1.000000


In [158]:
px.bar(food_related_hashtags_df.head(30),
       y='hashtags', x='users_count', height=700)

In [159]:
# food_related_hashtags_df.to_csv("hashtag_list.csv", index=False)

Tính phần trăm các video có hashtag về ẩm thực của từng user.


In [160]:
# Danh sách các hashtag liên quan đến ẩm thực
# food_related_hashtags = {
#     "reviewanngon", "ancungtiktok", "foodreview", "monngon", "anvat",
#     "foodtour", "eatclean", "buacomgiadinh", "amthuc", "diadiemanuong",
#     "reviewanuongsaigon", "anvatsaigon", "saigonfood"
# }

# Bước 1: Lấy ra 100 hashtag ẩm thực có nhiều video nhất
food_related_hashtags = food_related_hashtags_df.head(50)[
    'hashtags'].to_list()
# Bước 2: Tạo cột mới để đánh dấu video có chứa ít nhất một hashtag liên quan đến ẩm thực
filt_stats_df["is_food_related"] = filt_stats_df["hashtags"].apply(
    lambda tags: any(tag in food_related_hashtags for tag in tags)
)

# Bước 3: Tính tổng số video & số video liên quan đến ẩm thực của mỗi user
food_user_df = filt_stats_df.groupby("username").agg(
    total_videos=("username", "count"),
    food_related_videos=("is_food_related", "sum")
).reset_index()

# Bước 4: Tính phần trăm số video ẩm thực
food_user_df["food_video_percentage"] = (
    food_user_df["food_related_videos"] / food_user_df["total_videos"])

food_user_df = food_user_df.rename(columns={"username": "username"})
food_user_df.head()

,username,total_videos,food_related_videos,food_video_percentage
0,.banhsukem,34,0,0.000000
1,05061993chomthao,11,0,0.000000
2,1108ng,66,66,1.000000
3,1phutsaigon,63,8,0.126984
4,30days__giamcanlanhmanh,51,0,0.000000


Merge với `user_stats_df_0`.


In [161]:
user_stats_df = user_stats_df_0.merge(percent_vi, on='username').merge(frequency_df, on="username").merge(
    food_user_df, on=['username', 'total_videos'])
user_stats_df = user_stats_df[user_stats_df['frequency'] != 0]
user_stats_df.head()

display(user_stats_df.sample(5))

,username,total_views,avg_views,total_likes,avg_likes,total_comments,total_shares,total_collects,total_videos,followers,likes,engagement_rate,total_engagement,vi_video_percentage,days_between_videos,frequency,food_related_videos,food_video_percentage
103,beohayanvat,1301753.0,30994.119048,44847.0,1067.785714,921.0,10837.0,14170.0,42,61700.0,14500.0,0.054369,70775.0,0.997368,2.064210,0.484447,41,0.976190
571,quinnyandfelipene,3132527.0,62650.540000,218876.0,4377.520000,1897.0,20261.0,5547.0,50,15300.0,111.0,0.078716,246581.0,0.984615,1.508209,0.663038,1,0.020000
335,huygatuyen1,2907.0,581.400000,81.0,16.200000,1.0,1.0,8.0,5,161200.0,68.0,0.031304,91.0,0.564516,17.026053,0.058734,0,0.000000
540,phamtrang.2401,3772489.0,121693.193548,170097.0,5487.000000,718.0,11323.0,15550.0,31,304800.0,48600.0,0.052403,197688.0,1.000000,1.994479,0.501384,27,0.870968
156,cheducanh04,31745500.0,721488.636364,1524315.0,34643.522727,24748.0,75373.0,62786.0,44,116400.0,7461.0,0.053148,1687222.0,0.993631,2.418455,0.413487,19,0.431818


### Lọc user theo các ngưỡng


#### Visualization


In [162]:
px.histogram(user_stats_df, x='vi_video_percentage',
             title="Phân phối User theo Phần trăm video có caption tiếng Việt")

In [163]:
px.histogram(user_stats_df, x='food_video_percentage',
             title="Phân phối User theo Phần trăm video liên quan đến Ẩm thực", nbins=20)

In [164]:
px.histogram(user_stats_df, x='days_between_videos',
             title="Phân phối User theo Khoảng thời gian giữa các bài đăng")

In [165]:
px.histogram(user_stats_df, x='engagement_rate',
             title="Phân phối User theo Tỷ lệ tương tác")

In [166]:
px.histogram(user_stats_df, x='avg_views',
             title="Phân phối User theo Lượt xem trung bình")

In [167]:
px.histogram(user_stats_df, x='followers',
             title="Phân phối User theo Followers")

In [168]:
px.histogram(user_stats_df, x='likes',
             title="Phân phối User theo Likes")

#### Lọc


In [ ]:
print(user_stats_df.shape)
user_stats_df[['username', 'avg_views', 'total_videos', 'followers',
               'engagement_rate', 'frequency',
               'food_video_percentage', 'vi_video_percentage']].describe()

(758, 18)


,avg_views,total_videos,followers,engagement_rate,frequency,food_video_percentage,vi_video_percentage
count,7.580000e+02,758.000000,7.580000e+02,758.000000,758.000000,758.000000,758.000000
mean,4.933123e+05,62.930079,7.136357e+05,0.044407,1.187098,0.484228,0.854491
std,8.508811e+05,218.589550,2.393493e+06,0.040250,8.445511,0.419749,0.311281
min,2.208462e+02,2.000000,6.100000e+02,0.002140,0.031500,0.000000,0.000000
25%,6.695739e+04,20.000000,6.895000e+04,0.026515,0.249943,0.000000,0.947116
50%,2.039770e+05,36.000000,2.031500e+05,0.038582,0.461509,0.484360,0.991684
75%,5.214747e+05,61.000000,6.185750e+05,0.055122,0.835091,0.946637,1.000000
max,9.288571e+06,4745.000000,5.540000e+07,0.921040,194.813980,1.000000,1.000000


In [201]:
# Lọc lại danh sách user
# users_by_threshold = user_stats_df[
#     # (df_grouped["total_videos"] >= 4) &
#     (
#         (user_stats_df["food_video_percentage"]
#          >= min_food_perc_threshold)
#         &
#         (user_stats_df["frequency"]
#          >= min_freq_threshold)
#         &
#         (user_stats_df["engagement_rate"]
#          >= min_eng_rate_threshold)
#         &
#         (user_stats_df["avg_views"]
#          >= min_avg_views_threshold)
#     )
# ]

print(user_stats_df.shape)

# Bước 0: Lọc theo vi_video_percentage
min_vi_perc_threshold = 0.90
users_by_threshold = user_stats_df[
    user_stats_df["vi_video_percentage"] >= min_vi_perc_threshold
]
print(f"Min Vietnamese Video Percentage Threshold: {min_vi_perc_threshold}")
print(users_by_threshold.shape)

# Bước 1: Lọc theo food_video_percentage
# min_food_perc_threshold = users_by_threshold["food_video_percentage"].quantile(
#     0.6)
min_food_perc_threshold = 0.75
users_by_threshold = users_by_threshold[
    users_by_threshold["food_video_percentage"] >= min_food_perc_threshold
]
print(f"Min Food Video Percentage Threshold: {min_food_perc_threshold}")
print(users_by_threshold.shape)

# # Bước 2: Lọc tiếp theo avg_views
# min_avg_views_threshold = users_by_threshold["avg_views"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["avg_views"] >= min_avg_views_threshold
# ]
# print(f"Min Average Views Threshold: {min_avg_views_threshold}")
# print(users_by_threshold.shape)

# # Bước 3: Lọc tiếp theo engagement_rate
# min_eng_rate_threshold = users_by_threshold["engagement_rate"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["engagement_rate"] >= min_eng_rate_threshold
# ]
# print(f"Min Engagement Rate Threshold: {min_eng_rate_threshold}")
# print(users_by_threshold.shape)

# # Bước 4: Lọc cuối cùng theo frequency
# min_freq_threshold = users_by_threshold["frequency"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["frequency"] >= min_freq_threshold
# ]
# print(f"Min Frequency Threshold: {min_freq_threshold}")
# print(users_by_threshold.shape)

users_by_threshold.sort_values(by="total_videos", ascending=True)

(758, 18)
Min Vietnamese Video Percentage Threshold: 0.9
(601, 18)
Min Food Video Percentage Threshold: 0.75
(264, 18)


,username,total_views,avg_views,total_likes,avg_likes,total_comments,total_shares,total_collects,total_videos,followers,likes,engagement_rate,total_engagement,vi_video_percentage,days_between_videos,frequency,food_related_videos,food_video_percentage
95,be_mup1,2487000.0,1.243500e+06,74100.0,37050.000000,2392.0,6406.0,3805.0,2,23900.0,862.0,0.034862,86703.0,1.000000,15.751172,0.063487,2,1.000000
625,tebetravelmade,3695.0,1.847500e+03,81.0,40.500000,0.0,13.0,20.0,2,55100.0,398.0,0.030853,114.0,0.984064,7.479332,0.133702,2,1.000000
596,seonguk_vn,332300.0,1.661500e+05,12438.0,6219.000000,136.0,79.0,169.0,2,310500.0,2449.0,0.038586,12822.0,1.000000,2.166944,0.461479,2,1.000000
313,homnay_tuiangi,165500.0,8.275000e+04,2074.0,1037.000000,48.0,312.0,158.0,2,146900.0,19800.0,0.015662,2592.0,1.000000,24.858767,0.040227,2,1.000000
613,sun111992,3720.0,1.240000e+03,83.0,27.666667,13.0,11.0,11.0,3,13300.0,886.0,0.031720,118.0,1.000000,26.823472,0.037281,3,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,dung.thch.n,47589200.0,3.837839e+05,1289188.0,10396.677419,11668.0,82123.0,53940.0,124,148100.0,415.0,0.030194,1436919.0,0.996109,0.676288,1.478661,121,0.975806
753,wottueat,34471500.0,2.223968e+05,643043.0,4148.664516,12243.0,105954.0,34381.0,155,37200.0,2616.0,0.023081,795621.0,0.994226,0.719329,1.390185,155,1.000000
671,tinhtungtangvn,44916300.0,2.673589e+05,1387075.0,8256.398810,33578.0,628544.0,344698.0,168,296000.0,5446.0,0.053297,2393895.0,1.000000,0.651698,1.534452,164,0.976190
712,tunauan68,74496668.0,3.156638e+05,1842283.0,7806.283898,119717.0,51243.0,127426.0,236,275400.0,18.0,0.028735,2140669.0,0.991859,0.609479,1.640745,226,0.957627


In [197]:
video_info_df[video_info_df['author.uniqueId'].isin(
    users_by_threshold['username'])].shape

(71504, 81)

In [ ]:
def write_list_to_file(file_path, data_list):
    try:
        with open(file_path, "w", encoding="utf-8") as file:
            for item in data_list:
                file.write(str(item) + "\n")
        print(f"Successfully wrote {len(data_list)} items to {file_path}")
    except Exception as e:
        print(f"Error writing to file: {e}")


write_list_to_file('data/lists/final_users_list.txt',
                   list(users_by_threshold['username'].unique()))

Successfully wrote 264 items to data/lists/final_users_list.txt


### Lọc bằng cách tính điểm


In [173]:

def compute_score(df, columns_weights):
    """
    Tính điểm tổng hợp cho DataFrame user_stats_df với các cột và trọng số linh hoạt.

    Công thức tính điểm: 
      score = sum(weight_i * norm_column_i)
    """
    df_copy = df.copy()

    for col in columns_weights.keys():
        df_copy[col] = df_copy[col].fillna(0).clip(lower=0)

    max_values = {col: df_copy[col].max(
    ) or 1 for col in columns_weights.keys()}

    for col in columns_weights.keys():
        df_copy[f"norm_{col}"] = df_copy[col] / max_values[col]

    df_copy["score"] = sum(
        columns_weights[col] * df_copy[f"norm_{col}"]*100
        for col in columns_weights.keys()
    )

    return df_copy

In [174]:

# Ví dụ sử dụng:
columns_weights = {
    "food_video_percentage": 0.35,
    "engagement_rate": 0.05,
    "avg_views": 0.35,
    "frequency": 0.1,
    "followers": 0.35
}
# columns_weights = {
#     "avg_views": 0.35,
#     "engagement_rate": 0.25,
#     "followers": 0.35,
#     "frequency": 0.05
# }

# Tính điểm cho DataFrame
# users_by_score = compute_score(
#     user_stats_df[user_stats_df['vi_video_percentage'] >= 0.8], columns_weights)
# users_by_score = compute_score(users_by_threshold, columns_weights)
users_by_score = compute_score(
    users_by_threshold, columns_weights)

# In kết quả
users_by_score.sort_values('score', ascending=False)

# Sau đó bạn có thể:
users_by_score = users_by_score.sort_values(
    "score", ascending=False, ignore_index=True)

uesr_by_score_top = users_by_score.head(len(users_by_threshold))
display(users_by_score[["username", "score"]].merge(user_stats_df[[
        'followers', 'username', 'avg_views', 'frequency', 'engagement_rate']]))

,username,score,followers,avg_views,frequency,engagement_rate
0,tocvang.hoe,101.145690,2400000.0,2.207668e+06,0.499038,0.025528
1,haidangrevieww,90.653633,2300000.0,1.948041e+06,0.327733,0.043306
2,eatwpeach,88.330860,3000000.0,7.112327e+05,0.755843,0.052710
3,bon.tq1,87.307864,1300000.0,2.152925e+06,0.514905,0.025223
4,hoanganhdeptrailam,85.373161,909900.0,2.211576e+06,0.464031,0.041203
...,...,...,...,...,...,...
178,cundaily216,37.930596,9140.0,1.945481e+05,0.640267,0.051364
179,nunloangquang,37.924290,51300.0,8.537100e+04,0.336914,0.067560
180,thuntuoither,37.858267,254100.0,1.160400e+05,0.438054,0.069669
181,nartman.review,37.539055,69900.0,2.647693e+05,0.287567,0.034467


In [175]:
user_stats_df.columns

Index(['username', 'total_views', 'avg_views', 'total_likes', 'avg_likes',
       'total_comments', 'total_shares', 'total_collects', 'total_videos',
       'followers', 'likes', 'engagement_rate', 'total_engagement',
       'vi_video_percentage', 'days_between_videos', 'frequency',
       'food_related_videos', 'food_video_percentage'],
      dtype='object')

In [176]:
import json


def calculate_entropy_weights(df, columns_to_maximize):
    """
    Tính trọng số (weights) cho mỗi cột trong DataFrame df dựa trên phương pháp Entropy Weighting.
    """
    # 1) Chuẩn hóa dữ liệu về [0..1]
    df_normalized = df[columns_to_maximize].copy()
    for col in columns_to_maximize:
        df_normalized[col] = (df[col] - df[col].min()) / \
            (df[col].max() - df[col].min())

    # 2) Tính entropy cho mỗi cột
    n = len(df)  # số lượng user
    p = df_normalized / df_normalized.sum(axis=0)  # xác suất
    entropy = - (p * np.log(p + 1e-9)).sum(axis=0) / np.log(n)  # tránh log(0)

    # 3) Tính trọng số (weights) cho mỗi cột
    weights = (1 - entropy) / (1 - entropy).sum()

    # Trả về DataFrame với trọng số
    weights_df = pd.DataFrame(
        weights, columns=['weight'], index=columns_to_maximize)

    return weights_df


columns_to_maximize = ['total_views', 'avg_views', 'total_videos', 'total_engagement', 'followers',
                       # 'total_likes', 'avg_likes', 'total_comments', 'total_shares', 'total_collects', 'cluster', 'days_between_videos',
                       'engagement_rate', 'frequency', 'food_related_videos', 'food_video_percentage']


# Tính trọng số cho mỗi cột
weights_df = calculate_entropy_weights(
    user_stats_df, columns_to_maximize).sort_values("weight")
display(weights_df)
# Xem kết quả
columns_weights = json.loads(weights_df.to_json())['weight']

,weight
engagement_rate,0.019139
food_video_percentage,0.044749
food_related_videos,0.071535
avg_views,0.073959
total_videos,0.094550
followers,0.112524
total_views,0.181633
frequency,0.196737
total_engagement,0.205173


In [177]:
# Tính điểm cho DataFrame
df_with_scores = compute_score(user_stats_df, columns_weights)

# In kết quả
df_with_scores.sort_values('score', ascending=False)

# Sau đó bạn có thể:
user_stats_df_scored = df_with_scores.sort_values(
    "score", ascending=False)
user_stats_df_scored[["username", "score"]].head(100)  # Xem top 20

,username,score
432,metropolesoficial,58.953732
480,neko.nailstudio,33.072226
6,60giay.com,32.849961
637,thao_denn,22.590489
94,bayashi.tiktok,21.672103
...,...,...
337,huynhanhtuan_dienvien,6.491673
403,linhduyen171,6.472876
223,donghyeontv,6.472573
261,giang.ghin.chill,6.465658


In [178]:
# from sklearn.cluster import KMeans
# columns_to_consider = ['total_views', 'avg_views', 'total_likes', 'avg_likes',
#                        'total_comments', 'total_shares', 'total_collects',
#                        'total_videos', 'engagement_rate', 'days_between_videos',
#                        'food_related_videos', 'food_video_percentage', 'frequency']
# # Sử dụng các cột để phân nhóm (có thể chuẩn hóa dữ liệu nếu cần)
# X = user_stats_df[columns_to_consider]

# # Áp dụng KMeans (giả sử chia thành 3 nhóm)
# kmeans = KMeans(n_clusters=3)
# user_stats_df['cluster'] = kmeans.fit_predict(X)

# # Lấy các centroid của mỗi nhóm
# centroids = kmeans.cluster_centers_

# # Các cột quan trọng có thể được xác định từ các giá trị lớn trong centroids
# centroid_df = pd.DataFrame(centroids, columns=columns_to_consider)
# centroid_df

In [179]:
# def get_pareto_front(df, columns_maximize):
#     """
#     Tìm Pareto front trong DataFrame df, coi tất cả cột trong columns_maximize
#     đều là "càng lớn càng tốt".

#     Trả về sub-DataFrame gồm các hàng không bị chi phối bởi bất kỳ hàng nào khác.
#     """
#     dominated = set()
#     n = len(df)

#     for i in range(n):
#         if i in dominated:
#             # Hàng này đã bị chi phối, không cần check nữa
#             continue

#         # Kiểm tra xem có hàng j nào chi phối i không
#         for j in range(n):
#             if i == j:
#                 continue
#             # Nếu j >= i trên toàn bộ columns,
#             # và j > i trên ít nhất 1 cột => i bị chi phối
#             if (df.iloc[j][columns_maximize] >= df.iloc[i][columns_maximize]).all() \
#                and (df.iloc[j][columns_maximize] > df.iloc[i][columns_maximize]).any():
#                 dominated.add(i)
#                 break

#     # Các hàng không bị chi phối -> Pareto front
#     indices_front = [idx for idx in range(n) if idx not in dominated]
#     return df.iloc[indices_front].copy()

In [180]:
# # Tìm Pareto front
# pareto_df = get_pareto_front(user_stats_df, columns_to_maximize)

# print(f"Số user trên Pareto front: {len(pareto_df)}")
# pareto_df.head()

In [181]:
# pareto_df.columns